# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [18]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files
* checking your current working directory
* Get your current folder and subfolder event data
* Create a for loop to create a list of files and collect each filepath
* join the file path and roots with the subdirectories using glob

In [19]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables
* initiating an empty list of rows that will be generated from each file
    * for every filepath in the file path list
    * reading csv file
    * creating a csv reader object
    
    * extracting each data row one by one and append it
    
* uncomment the code if you would like to get total number of rows:  `print(len(full_data_rows_list))`
* uncomment the code if you would like to check to see what the list of event data rows will look like:
 `print(full_data_rows_list)`
 
* creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables


In [20]:
full_data_rows_list = [] 
    
for f in file_path_list:

 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 


csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


#### check the number of rows in your csv file

In [21]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile.png**</font> after the code above is run:<br>

<img src="pics/event_datafile.png">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster
* Make a connection to a Cassandra instance your local machine (127.0.0.1)

* To establish connection and begin executing queries, need a session

In [22]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [23]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
        """)
except Exception as e:
    print(e)

#### Set Keyspace

In [24]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1:  artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

* The table name: `song_info_by_sessionid`
* The columns in the table: `artist`, `song`, `length`, `sessionId`, `itemInSession`
* The primary key: partition key `sessionId`, and clustering key `itemInSession`


                    

#### Creating Table

In [25]:
query = "CREATE TABLE IF NOT EXISTS song_info_by_sessionid"
query += "(sessionId int, itemInSession int, artist text, song text, \
length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Insert Data
* Assign the INSERT statements into the `query` variable
* Assign which column element should be assigned for each column in the INSERT statement.

In [26]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_sessionid (sessionId, itemInSession, artist, song\
        , length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9]\
                                , float(line[5])))

#### SELECT to verify that the data have been inserted into each table

In [27]:
from prettytable import PrettyTable
x = PrettyTable()

In [28]:
query = "SELECT artist, song, length From song_info_by_sessionid \
        where sessionId = 338 AND itemInSession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    x.field_names = ["artist", "song", "length"]
    x.add_row(row)
print(x)

+-----------+---------------------------------+--------------------+
|   artist  |               song              |       length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


## Query 2: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

* The table name: `info_by_userid_sessionid`
* The columns in the table: `artist`, `song`, `itemInSession`, `firstName`, `lastName`, `userId`, `sessionId`
* The primary key: partition key `userId`, `sessionId`, and clustering key `itemInSession`

#### Creating Table
                    

In [29]:
query = "CREATE TABLE IF NOT EXISTS info_by_userid_sessionid"
query += "(userId int, sessionId int, itemInSession int, artist text, \
song text, firstName text, lastName text, \
PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Insert Data
* Assign the INSERT statements into the `query` variable
* Assign which column element should be assigned for each column in the INSERT statement.


In [30]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO info_by_userid_sessionid (userId, sessionId, itemInSession\
        , artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3])\
                                , line[0], line[9], line[1], line[4]))

 #### SELECT to verify that the data have been inserted into each table

In [33]:
query = "SELECT artist, song, itemInSession, firstName, lastName From \
        info_by_userid_sessionid where userId=10 AND sessionId=182"
y = PrettyTable()

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    y.field_names = ["artist", "song", "itemInSession", "firstName", "lastName"]
    y.add_row(row)
print(y)

+-------------------+------------------------------------------------------+---------------+-----------+----------+
|       artist      |                         song                         | itemInSession | firstName | lastName |
+-------------------+------------------------------------------------------+---------------+-----------+----------+
|  Down To The Bone |                  Keep On Keepin' On                  |       0       |   Sylvie  |   Cruz   |
|    Three Drives   |                     Greece 2000                      |       1       |   Sylvie  |   Cruz   |
| Sebastien Tellier |                      Kilometer                       |       2       |   Sylvie  |   Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |       3       |   Sylvie  |   Cruz   |
+-------------------+------------------------------------------------------+---------------+-----------+----------+


## Query 3: user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

* The table name: `user_history_by_song`
* The columns in the table: `firstName`, `lastName`, `song`, `userId`
* The primary key: partition key `song`, and clustering key `userId`
                  

#### Creating Table

In [34]:
query = "CREATE TABLE IF NOT EXISTS user_history_by_song"
query += "(song text, userId int, firstName text, lastName text, \
PRIMARY KEY ((song), userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Insert Data
* Assign the INSERT statements into the `query` variable
* Assign which column element should be assigned for each column in the INSERT statement.

In [35]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_history_by_song (song, userId, \
        firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"

        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### SELECT to verify that the data have been inserted into each table

In [36]:
query = "SELECT firstName, lastName, song, userId From \
        user_history_by_song WHERE song='All Hands Against His Own'"

z = PrettyTable()

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    z.field_names = ["firstName", "lastName", "song", "userId"]
    z.add_row(row)
print(z)

+------------+----------+---------------------------+--------+
| firstName  | lastName |            song           | userId |
+------------+----------+---------------------------+--------+
| Jacqueline |  Lynch   | All Hands Against His Own |   29   |
|   Tegan    |  Levine  | All Hands Against His Own |   80   |
|    Sara    | Johnson  | All Hands Against His Own |   95   |
+------------+----------+---------------------------+--------+


### Drop the tables before closing out the sessions

In [37]:
query = "DROP TABLE IF EXISTS song_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS user_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [38]:
session.shutdown()
cluster.shutdown()